In [1]:
import os
os.chdir('C:/Users/eddie/Desktop/Prof. Rumi Chunara/alcohol')
import pickle
import sklearn
print(sklearn.__version__)
import pandas
import numpy as np
import sys
from evaluation.metric import computeAccuracy, computeF1Score, computeAUC
# from sklearn.metrics import roc_auc_score

# %load_ext autoreload
# %autoreload
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from scipy.stats import uniform
# %reload_ext pipelines.alcohol.AlcoholPipeline
from pipelines.alcohol import AlcoholPipeline

0.17


In [2]:
clf_alc_initial = pickle.load(open('./alcohol classifiers/clf_alc_UPDATED.p', 'rb'))
# clf_fpa    = pickle.load(open('./alcohol classifiers/clf_fpa_UPDATED.p', 'rb'))
# clf_fpl_dl = pickle.load(open('./alcohol classifiers/clf_fpl_double_labeled', 'rb'))
df = pandas.read_csv("./data/alcohol_training_instances.csv")


In [38]:
clf_alc_initial.steps
# with open('structure.txt', 'w') as f:
#     tmp = ''.join(str(e) for e in clf_alc_initial.steps)
#     f.write(tmp)

[('features', FeatureUnion(n_jobs=1,
         transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=71233, min_df=1,
     ...      token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
          vocabulary=None))]))],
         transformer_weights=None)),
 ('scaler', Normalizer(copy=True, norm='l2')),
 ('clf',
  LogisticRegression(C=177.08103265690417, class_weight=None, dual=False,
            fit_intercept=True, intercept_scaling=1, max_iter=100,
            multi_class='ovr', n_jobs=None, penalty='l2', random_state=None,
            solver='liblinear', tol=0.000655077907893521, verbose=0,
            warm_start=None))]

Prepare the alcohol data

In [39]:
alc_labels = []
# extracting pre-labeled classification
for i in range(0,len(df.labels)):
    alc_labels.append(df.labels[i][12])
alc_labels = list(map(int, alc_labels))
print(alc_labels[:10])
print(len(alc_labels))
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, alc_labels, test_size=0.1, random_state=26)

[0, 1, 1, 1, 1, 1, 1, 1, 0, 1]
15651


First we check the initial model performance

In [40]:
clf_alc_initial.fit(X_train, y_train)
# predict in testing set.
y_alc_initial = clf_alc_initial.predict(X_test)
# print('length of testing set: ', len(y_alc_initial))
computeAccuracy(y_alc_initial, y_test)
f1 = computeF1Score(y_test, y_alc_initial)
print('F1 Score: ', f1)
auc_score_initial = computeAUC(clf_alc_initial, X_test, y_test, plot=False)
# auc_score = computeAUC(clf_alc_LR_updateParams, X_train, y_train, X_test, y_test, plot=True, plotTitle="Alcohol ROC Curve")
print('alcohol LR AUC: ', auc_score_initial)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


length of total comps:  1566
pretrained alcohol classifer:  261 different item in whole dataset
accuracy: 83.333333%
F1 Score:  0.8718703976435936
alcohol LR AUC:  0.8722371835589783


Then we make our own SVM model and update structure parameters in SVM model. To make sure they become same with the initial model

In [41]:
clf_alc_SVM = AlcoholPipeline(global_features=["text"]).pipeline(LinearSVC())
params_LR = {'features__text__tfidf__dtype':np.int64, 'features__text__tfidf__max_features':71233, 'features__text__tfidf__ngram_range':(2,5) 
            }
# to see all parameters in model
# clf_alc_LR_updateParams.get_params().keys()
clf_alc_SVM.set_params(**params_LR)
clf_alc_SVM.steps

[('features', FeatureUnion(n_jobs=1,
         transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=71233, min_df=1,
     ...      token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
          vocabulary=None))]))],
         transformer_weights=None)),
 ('scaler', Normalizer(copy=True, norm='l2')),
 ('clf', LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
       intercept_scaling=1, loss='squared_hinge', max_iter=1000,
       multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
       verbose=0))]

Train the SVM model (with updated parameters), and predict in testing set

In [42]:
clf_alc_SVM.fit(X_train, y_train)
# predict in testing set.
y_alc_SVM = clf_alc_SVM.predict(X_test)
# print('length of testing set: ', len(y_alc_initial))
computeAccuracy(y_alc_SVM, y_test)
f1_score_SVM = computeF1Score(y_test, y_alc_SVM)
print('F1 Score: ', f1_score_SVM)
auc_score_SVM = computeAUC(clf_alc_SVM, X_test, y_test, plot=False)
# auc_score = computeAUC(clf_alc_LR_updateParams, X_train, y_train, X_test, y_test, plot=True, plotTitle="Alcohol ROC Curve")
print('alcohol LR AUC: ', auc_score_SVM)
# clf_alc_SVM.steps

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


length of total comps:  1566
pretrained alcohol classifer:  231 different item in whole dataset
accuracy: 85.249042%
F1 Score:  0.887481734047735
alcohol LR AUC:  0.8839735210587377


We can see the SVM model we build with updated parameters achieve even better results than the initial model, compared to initial model with 0.87 in F1 score and 0.87 in AUC score.